In [20]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import math
import csv

In [21]:
TIME_THRESHOLD =10

In [22]:
data_path = 'training_files/'

In [23]:
#-------------------------------------------------------------------------------

In [24]:
df = pd.read_csv('training_files/user20/session_9673196280')
df

,record timestamp,client timestamp,button,state,x,y
0,0.000,0.000,NoButton,Move,708,386
1,0.124,0.125,NoButton,Move,707,399
2,0.229,0.234,NoButton,Move,707,362
3,0.345,0.344,NoButton,Move,704,359
4,0.712,0.437,NoButton,Move,702,368
...,...,...,...,...,...,...
40470,13294.976,13295.061,NoButton,Move,272,624
40471,13295.259,13295.154,NoButton,Move,267,622
40472,13295.259,13295.342,NoButton,Move,265,622
40473,13295.411,13295.498,NoButton,Move,264,622


In [25]:
#-------------------------------------------------------------------------------

In [26]:
def remove_duplicate(df):
    df = df.drop_duplicates().reset_index(drop=True)
    return df
    
    

In [27]:
# upper_bound = 2000
# lower_bound = 0
# def remove_out_of_bound(df):
#     df = df[df['x']<=upper_bound]
#     df = df[df['y']<=upper_bound]
#     df = df[df['x']>=lower_bound]
#     df = df[df['y']>=lower_bound]
#     return df.reset_index(inplace=True)

In [28]:
upper_bound = 2000
lower_bound = 0

def process_out_of_bound(df):
    
    mean_x, mean_y = df.x.mean(), df.y.mean()
    if (df.iloc[0][4] not in range(lower_bound, upper_bound) or df.iloc[0][5] not in range(lower_bound, upper_bound)):
        df['x'][0]=mean_x
        df['y'][0]=mean_y
    
    for idx, row in df.iterrows():
        if idx<=0:
            continue
        if (row['x'] not in range(lower_bound, upper_bound) or row['y'] not in range(lower_bound, upper_bound)):
            # print("-------------------------------------------")
            # print(idx, row['x'], row['y'])
            tmp_idx = None
            for i in range(idx-1,-1,-1):
                if (df.iloc[i][4] in range(lower_bound, upper_bound) and df.iloc[i][5] in range(lower_bound, upper_bound)):
                    tmp_idx = i
                    break
                    #print("tmp: ", tmp_idx)
            df.loc[idx, 'x'] = df.loc[tmp_idx, 'x']
            df.loc[idx, 'y'] = df.loc[tmp_idx, 'y']

    return df.reset_index(drop = True)


            



In [29]:
def drop_scroll_button(df):
    return df.drop(df[df.button=='Scroll'].index).reset_index(drop=True)

In [30]:
def assign_action_label(df, start, value):
    

    if value=='PC':
        df.loc[start, 'Action'] = value
        for i in range(start-1, -1, -1):
            if df.loc[i, 'state'] =='Released':
                break
            df.loc[i, 'Action'] = value
    elif value == 'DD':
        for i in range(start, -1, -1):
            df.loc[i, 'Action'] = value
            if df.loc[i, 'state'] == 'Pressed':
                # print("i: ", i)
                break

    return df



In [31]:
def segment_event(df):

    if df['state'][0]=="Released":
        df.loc[0, 'state']='Move'

    list_released_idx = []
    for idx, row in df.iterrows():
        if row['state'] =='Released':
            list_released_idx.append(idx)
    for ele in list_released_idx:
        if df.iloc[ele-1][3]=="Pressed":
            df = assign_action_label(df, ele, 'PC')
        else:
            df = assign_action_label(df, ele, 'DD')
    
    #process in case data are redundant at the end of the df (without "Released" at the end)
    if list_released_idx[-1]<len(df):
        remaining_actions = df.loc[list_released_idx[-1]+1:, :]['state'].unique().tolist()
        if "Move" in remaining_actions and "Pressed" in remaining_actions and "Drag" in remaining_actions:
            for i in range(list_released_idx[-1]+1, len(df)):
                if df.loc[i, 'state']=="Move":
                    df.loc[i, 'Action'] = 'MM'
                else:
                    df.loc[i, 'Action'] = 'DD'
        else: #in case the rest only has Move and Pressed
            for i in range(list_released_idx[-1]+1, len(df)):
                df.loc[i, 'Action'] = 'MM'


    #in case Action are still not assign any action
    for i in range(len(df)):
        if df.loc[i, 'Action'] in ["PC", "DD", "MM"]:
            continue
        else:
            df.loc[i, 'Action']="MM"

     



    return df
                


In [32]:

def threshold_PC(df):

    idx = len(df)-1
    is_break = None
    while idx>=0:
        is_break = False
        
        while df.loc[idx, 'Action'] != 'PC':
            idx -=1
            

        if (idx>1) and (df.loc[idx, 'state'] == 'Move' and df.loc[idx-1, 'state']=='Move'):
            delta_time = df.loc[idx, 'client timestamp'] - df.loc[idx-1, 'client timestamp']
            if delta_time >= TIME_THRESHOLD:

                for i in range(idx-1, -1, -1):
                    # if df.loc[i, 'Action'] != 'PC' or ((df.loc[i, 'client timestamp']-df.loc[i-1, 'client timestamp'])>= TIME_THRESHOLD)\
                    #     or df.loc[i, 'state']!= 'Move':
                        
                    #     idx=i
                    #     break
                    if df.loc[i, 'state'] != 'Move':
                        idx = i
                        is_break = True
                        break
                    df.loc[i, 'Action'] = 'MM'
        if not is_break:
            idx -=1


    return df


In [33]:

def divide_action(df):

    flag = 0
    #is_break = False
    the_last_action = None
    the_last_state = None

    idx = len(df)-1
    in_MM = None
    while idx>=0:
        #is_break = False

        in_MM = False
        row = df.loc[idx]

        curr_action = row['Action']
        if curr_action != 'MM':
            if the_last_action!= curr_action or (df.loc[idx, 'state']=='Released'):
                # ((df.loc[idx, 'state'] != the_last_state) and (df.loc[idx, 'state']=='Released')):
                flag = int(not flag)
                the_last_action = df.loc[idx, 'Action']
            df.loc[idx, 'Flag'] = flag
        else:

            ii = idx
            flag = int(not flag)
            is_break = None
            while(df.loc[ii, 'Action'] =='MM' and ii>1):
                is_break=False
                df.loc[ii, 'Flag'] = flag

                delta_time = df.loc[ii, 'client timestamp'] - df.loc[ii-1, 'client timestamp']
                if delta_time >= TIME_THRESHOLD:
                    flag = int (not flag)
                    for j in range(ii-1, -1, -1):
                        if df.loc[j, 'Action'] != 'MM':
                            ii=j
                            is_break = True
                            flag= int( not flag)
                            break
                        
                        df.loc[j, 'Flag'] = flag
                        if ((df.loc[j, 'client timestamp']-df.loc[j-1, 'client timestamp'])>= TIME_THRESHOLD):
                            ii = j-1
                            flag = int (not flag)
                            is_break = True
                            break
                if not is_break:
                    ii -=1
            
            the_last_action = 'MM'
            idx =ii
            in_MM = True
        
        if not in_MM:
            idx -=1


    return df


In [34]:

def indices_segment_event(df):
    curr = df.loc[0, 'Flag']
    list_indices = []

    for i in range(len(df)):
        if df.loc[i, 'Flag'] != curr:
            list_indices.append(i)
            curr = df.loc[i, 'Flag']
    
    return list_indices


In [35]:
def preprocessing(df):
    df = remove_duplicate(df)
    df = process_out_of_bound(df)
    df = drop_scroll_button(df)
    df = segment_event(df)
    df = threshold_PC(df)
    df = divide_action(df)
    return df


In [36]:
# df = preprocessing(df)
# df

In [37]:
X = [0]
Y = [0]
t = [0]

In [38]:
S = None
list_indices = None

In [39]:

def get_length_trajectory(df, i, delta_distance):
    if i in list_indices:
        S[i]=0
    else:
        S[i]= S[i-1]+delta_distance
    
    return S[i]
    


In [40]:
def remove_action_less_than_4actions(df, list_indices):
    list_indices = list_indices[-1::-1]
    list_indices.append(0)
    for i, ele in enumerate(list_indices):
        if i ==0:
            break
        len_acts = ele - list_indices[i-1] +1 
        if len_acts <= 4:
            df = df.drop(df.loc[list_indices[i-1]:ele-1].index)
    return df.reset_index(drop=True)
        

In [41]:

epsilon = 10e-4
def calculate_velocity(df):
    global S, list_indices
    S = np.zeros((len(df)))
    list_indices = indices_segment_event(df)


    for i in range(1, len(df)):
        delta_x = df.loc[i, 'x'] - df.loc[i-1, 'x'] + epsilon
        delta_y = df.loc[i, 'y'] - df.loc[i-1, 'y'] + epsilon
        delta_t = df.loc[i, 'client timestamp'] - df.loc[i-1, 'client timestamp'] + 10e-6

        X.append(delta_x)
        Y.append(delta_y)
        t.append(delta_t)

        vx = delta_x/delta_t
        vy = delta_y/delta_t
        vxy = math.sqrt(pow(vx,2)+pow(vy,2))

        #--------caculate the angle velocity------------
        if delta_x==0:
            delta_alpha = math.pi/2
        else:
            delta_alpha = np.arctan(delta_y/delta_x)
        v_alpha = delta_alpha/delta_t
        acceleration = vxy/delta_t
        jerk = acceleration/delta_t

        delta_distance = math.sqrt(pow(delta_x,2)+pow(delta_y,2))
        try:
            curvature_time = delta_alpha/delta_distance
        except:
            print(i)
            print(df.loc[i, 'x'], df.loc[i-1, 'x'], df.loc[i, 'y'], df.loc[i-1, 'y'])
            print(delta_distance)
            print(delta_x)
            print(delta_y)
            print(1/0)

        trajectory_length = get_length_trajectory(df, i, delta_distance)

        df.loc[i, 'Vx'] = abs(vx)
        df.loc[i, 'Vy'] = abs(vy)
        df.loc[i, 'Vxy'] = vxy
        df.loc[i, 'V_alpha'] = v_alpha
        df.loc[i, 'Accelerate'] = abs(acceleration)
        df.loc[i, 'Jerk'] = abs(jerk)
        df.loc[i, 'Curvature_time'] = abs(curvature_time)
        df.loc[i, 'Trajectory_length'] = trajectory_length

    df.loc[0, 'Vx'] = 0
    df.loc[0, 'Vy'] = 0
    df.loc[0, 'Vxy'] = 0
    df.loc[0, 'V_alpha'] = 0
    df.loc[0, 'Accelerate'] = 0
    df.loc[0, 'Jerk'] = 0
    df.loc[0, 'Curvature_time'] = 0
    df.loc[0, 'Trajectory_length'] = 0

    return df

    

In [42]:
# df = calculate_velocity(df)

In [43]:
# df = remove_action_less_than_4actions(df, list_indices)

In [44]:
def get_action_label(action):
    if action == 'MM':
        return 1
    elif action == 'PC':
        return 2
    else:
        return 3

In [45]:
def write_csv(path, row):
    with open(path, 'a', encoding='UTF8') as f:
        writer = csv.writer(f)

        # write a row to the csv file
        writer.writerow(row)


In [46]:
def create_final_features(df, list_indices, des_path):
    list_indices.insert(0, 0)
    for idx, ele in enumerate(list_indices):
        if idx ==0:
            continue
        sub_df = df.loc[list_indices[idx-1]:ele-1].reset_index(drop=True)
        # sub_df = sub_df[sub_df['state']!= 'Pressed']
        # sub_df = sub_df[sub_df['state']!= 'Released']

        avg_vx, a_vy, avg_v, avg_valpha, avg_accelerate, avg_jerk, avg_curvature = sub_df['Vx'].mean(), sub_df['Vy'].mean(), sub_df['Vxy'].mean(), \
                                                                                    sub_df['V_alpha'].mean(), sub_df['Accelerate'].mean(), sub_df['Jerk'].mean(),\
                                                                                    sub_df['Curvature_time'].mean()
        trajectory_length = sub_df.loc[len(sub_df)-1, 'Trajectory_length']

        action_type = get_action_label(sub_df.loc[0, 'Action'])

        number_of_points = len(sub_df)

        one_row = np.array([avg_vx, a_vy, avg_v, avg_valpha, avg_accelerate, avg_jerk, avg_curvature, trajectory_length, int(number_of_points), int(action_type)])

        write_csv(des_path, one_row)


    

In [47]:
# create_final_features(df, list_indices)

In [48]:
source_path = './training_files/'
des_path = './train'

In [51]:
for user in os.listdir(source_path):
    user_path = os.path.join(source_path, user)
    for file in os.listdir(user_path):
        file_path = os.path.join(user_path, file)
        print(file_path)

        global S, list_indices
        S, list_indices = None, None

        X = [0]
        Y = [0]
        t = [0]
        
        df = pd.read_csv(file_path)
        df = preprocessing(df)
        df = calculate_velocity(df)
        df = remove_action_less_than_4actions(df, list_indices)

        user_des = os.path.join(des_path, user)
        if not os.path.isdir(user_des):
            os.mkdir(user_des)
        path = os.path.join(user_des, file+'.csv')
        create_final_features(df, list_indices, path)

./training_files/user23/session_9962419470
./training_files/user23/session_7336984763
./training_files/user23/session_2080551697
./training_files/user23/session_1123244103
./training_files/user23/session_3195683016


KeyboardInterrupt: 

In [70]:
a = pd.read_csv('./features.csv', names=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'u'], index_col=False)
a

/tmp/ipykernel_13078/535360259.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  a = pd.read_csv('./features.csv', names=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'u'], index_col=False)


,a,b,c,d,e,f,g,h,u
0,37.249793,76.732306,94.829534,5605.760478,1.010910e+06,1.010153e+11,79.449512,0.0,2.0
1,159.889064,195.674483,274.355988,1481.900784,2.694403e+05,2.668330e+10,21.208013,0.0,2.0
2,654.313288,262.814728,714.853413,1.353296,6.195870e+03,5.427659e+04,74.104161,0.0,2.0
3,571.967463,210.684156,623.453217,5611.191431,1.015658e+06,1.010153e+11,79.351595,0.0,2.0
4,364.557710,97.428557,389.654077,4618.270780,8.353425e+05,8.318906e+10,32.874052,0.0,1.0
...,...,...,...,...,...,...,...,...,...
2456,0.807792,1.350925,2.147506,2.086091,7.217462e+00,2.581451e+01,0.615696,0.0,1.0
2457,13.960570,8.116161,17.361162,0.510050,1.290683e+02,9.686933e+02,185.451365,0.0,2.0
2458,109.555252,251.413022,320.435154,2616.802800,4.740196e+05,4.714047e+10,18.629237,0.0,1.0
2459,50.003017,53.017607,73.728287,39270.778029,7.071087e+06,7.071068e+11,278.465082,0.0,3.0


In [71]:
a= a.loc[1:4].reset_index(drop=True)
a

,a,b,c,d,e,f,g,h,u
0,159.889064,195.674483,274.355988,1481.900784,2.694403e+05,2.668330e+10,21.208013,0.0,2.0
1,654.313288,262.814728,714.853413,1.353296,6.195870e+03,5.427659e+04,74.104161,0.0,2.0
2,571.967463,210.684156,623.453217,5611.191431,1.015658e+06,1.010153e+11,79.351595,0.0,2.0
3,364.557710,97.428557,389.654077,4618.270780,8.353425e+05,8.318906e+10,32.874052,0.0,1.0


In [69]:
a.loc[-1, 'u']

KeyError: -1